In [1]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [2]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistory, FunctionCallContent, FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function

from typing import Annotated

In [3]:
import json
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import RealDictCursor
load_dotenv()

True

In [4]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id, 
                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                        deployment_name=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL"),
                                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                        ))
    return kernel

In [5]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()

In [6]:
from decimal import Decimal

class Accounts:
    def __init__(self):
        connection = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )
        self.connection = connection
        self.cursor = self.connection.cursor(cursor_factory=RealDictCursor)

    @kernel_function(description="Fetches information from a database for a given customer id.")
    def get_customer_info(self, customer_id: Annotated[str, "customer id to look up"]) -> Annotated[str, "Returns the customer information."]:
        """
        Fetches account information from the CustomerData table in PostgreSQL database.

        :param customer_id (int): ID of the account.
        :return: Customer information as a JSON string.
        :rtype: str
        """
        print("get_customer_info function called... ")
        try:
            connection = self.connection
            cursor = self.cursor
            cursor.execute("select * from CustomerData WHERE customer_id = %s", (customer_id,))
            customer_record = cursor.fetchone()
            if customer_record:
                # Convert Decimal values to strings
                for key, value in customer_record.items():
                    if isinstance(value, Decimal):
                        customer_record[key] = str(value)
                return json.dumps({"customer_info": customer_record})
            else:
                return json.dumps({"error": "Customer not found."})
        except Exception as e:
            return json.dumps({"error": str(e)})
        finally:
            if connection:
                cursor.close()
                connection.close()

In [7]:
acc = Accounts()
acc.get_customer_info(123456)

get_customer_info function called... 


'{"customer_info": {"customer_id": 123456, "card_blocked": true, "payment_due": true, "card_type": "Visa"}}'

In [8]:
def _create_kernel_with_chat_completion_and_plugin(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id, 
                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                        deployment_name=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL"),
                                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                        ))
    kernel.add_plugin(Accounts(), plugin_name="accounts_plugin")
    return kernel

In [9]:
TRANSCRIPTION_REVIEWER = "TranscriptionReviewer"
TRANSCRIPTION_REVIEWER_INSTRUCTIONS = """
You are a transcription reviewer who has been tasked with reviewing a conversation between a customer service agent and a customer.
The goal is to examine the conversation and determine if the customer's card is blocked. 
If so, extract the customer id from the transcript and provide it to the Orchestrator.
if not, provide a response stating that you are skiping this transcript.
"""

ANALYST_NAME = "BusinessAnalyst"
ANALYST_INSTRUCTIONS = """
You are a business analyst who has been tasked to query a table named CustomerData in a database.
The table has the following columns: customer_id, card_blocked, payment_due, and card_type.
If the card is blocked and payment is due, provide a response stating that the customer's card is blocked and payment is due.
If the card is blocked and payment is not due, provide a response stating that the card is blocked due to some other reason.
"""

ORCHESTRATOR_NAME = "Orchestrator"
ORCHESTRATOR_INSTRUCTIONS = """
You are an orchestrator who has been tasked with coordinating the work of the transcription reviewer and the business analyst.
You will receive the customer id from the transcription reviewer and pass it to the business analyst.
If the reviwer provides a response stating that the card is not blocked, you will provide a response stating that the transcript is being skipped.
After the analysis is provided by Business Analyst please approve the analysis using keyword "approved"
"""

TASK = "Read transcript and determine the reason for blocked card"

# if the business analyst provides a response stating that the card is blocked and payment is due, you will provide a response stating that the customer's card is blocked and payment is due.
# if the business analyst provides a response stating that the card is blocked due to some other reason, you will provide a response stating that the customer's card is blocked due to some other reason.


In [10]:
# 1. Create the reviewer agent based on the chat completion service
agent_reviewer = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion("TranscriptionReviewer"),
    name=TRANSCRIPTION_REVIEWER,
    instructions=TRANSCRIPTION_REVIEWER_INSTRUCTIONS,
)

In [11]:
# 2. Create the copywriter agent based on the chat completion service
agent_analyst = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion_and_plugin("BusinessAnalyst"),
    name=ANALYST_NAME,
    instructions=ANALYST_INSTRUCTIONS,
    
)

In [12]:
# 3. Create the reviewer agent based on the chat completion service
agent_orchestrator = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion("Orchestrator"),
    name=ORCHESTRATOR_NAME,
    instructions=ORCHESTRATOR_INSTRUCTIONS,
)

In [ ]:
# 4. Place the agents in a group chat with a custom termination strategy
group_chat = AgentGroupChat(
    agents=[
        agent_reviewer,
        agent_analyst,
        agent_orchestrator 
        
    ],
    termination_strategy=ApprovalTerminationStrategy( # other termination strategies
        agents=[agent_orchestrator],
        maximum_iterations=10,
    ),
)

In [14]:
from semantic_kernel.agents.group_chat.agent_chat import ChatMessageContent

message_content = ChatMessageContent(role="user", content=""" 
                                     Customer ID : 123456
Date: 16-03-2025 10:23:30

Call Transcript:
            
Agent: how can I help you today? 
Customer: My card is blocked, could you please help me with that?
Agent: I am sorry to hear that. Can you please provide me with your name, contact number, email id and address to unblock the card?
Customer: My name is Vikas, contact number is 1234567890, email id is vsdsdf@gmail.com and address is 1234, 5th Avenue, New York, NY 10001
Agent: Thank you for providing the details. I have raised a request for this to be looked at immediately. You will receive a confirmation email shortly. Is there anything else I can help you with?
Customer: No, that's all. Thank you.
Agent: You are welcome. Have a great day!
                                     
                                     """)

In [ ]:
# 4. Add the task as a message to the group chat
await group_chat.add_chat_message(message_content)


In [16]:
# 5. Invoke the chat
async for content in group_chat.invoke():
    print(f"# {content.name}: {content.content}")


# TranscriptionReviewer: The customer's card is blocked. 

Customer ID: 123456

I will provide this to the Orchestrator.
get_customer_info function called... 
# BusinessAnalyst: The customer's card is blocked, and payment is due.
# Orchestrator: The transcript will be processed since the card is blocked. 

I will now proceed with the analysis provided. 

Analysis: The customer's card is blocked, and payment is due.

Approved.
